In [1]:
from datasets import dog_dataset, cub_dataset, food_dataset, cub_and_dogs
from models.models_to_finetune import deit_small_patch16_224, myresnetv2_task1, myresnetv2_task2, myresnetv2_for_c_loss
import PIL
import numpy as np
from tqdm import tqdm
import torch
import torch.optim as optim
from torchvision import transforms
import config
import sys
import math
from loss import CenterLoss
from run_center_loss import train_model_with_closs
from vit.vit_pytorch.nest import NesT
import timm
from torch.optim.lr_scheduler import ReduceLROnPlateau


In [2]:
epochs = 50
batch_size = 32
mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)
test_transform=transforms.Compose([
                    transforms.Resize((448, 448)),
                    transforms.ToTensor(),
                    transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
                ])

data_transform4 = transforms.Compose([  #

        transforms.Resize((448, 448)),
        transforms.RandomRotation(20),
        transforms.GaussianBlur(3, sigma=(0.1, 2.0)),
        transforms.ToTensor(),
        transforms.Normalize(mean=mean, std=std)
    ])

In [3]:
train_loader, val_loader, test_loader = cub_and_dogs(bs=batch_size, data_transform=data_transform4, test_transform=test_transform)

In [4]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

model = myresnetv2_for_c_loss(num_classes=320)
model = model.to(device)

for param in model.parameters():
    param.requires_grad = True

my_list = ['head.1.weight', 'head.1.bias','head.3.weight', 'head.3.bias']
params = list(filter(lambda kv: kv[0] in my_list, model.named_parameters()))
base_params = list(filter(lambda kv: kv[0] not in my_list, model.named_parameters()))

crit_entr = torch.nn.CrossEntropyLoss()
crit_closs = CenterLoss(num_classes=320, feat_dim=512)
path = "/home/hashmat.malik/Fall 2021/CV703 Lab/Week5/datasets/Task2:cub_and_dogs/Exp1/model_best_resnet_v2_cubs_dogs_4.pth.tar"
checkpoint = torch.load(path)
model.load_state_dict(checkpoint['state_dict'])
optimizer = optim.Adam([{'params':  [i[1]for i in params], 'lr': 0.0001, 'betas': (0.5, 0.999)},
                {'params':  [i[1]for i in base_params], 'lr': 0.00001, 'betas': (0.5, 0.999)},
                {'params': crit_closs.parameters(), 'lr': 0.01, 'betas': (0.5, 0.999)}
                        ])

scheduler = ReduceLROnPlateau(optimizer, 'max')
train_model_with_closs(60, train_loader, val_loader, test_loader, optimizer, scheduler, crit_entr, crit_closs, model, f'resnet_v2_closs_new_lr_{0.01}', is_train=False)


/home/hashmat.malik/Fall 2021/CV703 Lab/Week5/loss.py:38: UserWarning: This overload of addmm_ is deprecated:
	addmm_(Number beta, Number alpha, Tensor mat1, Tensor mat2)
Consider using one of the following signatures instead:
	addmm_(Tensor mat1, Tensor mat2, *, Number beta, Number alpha) (Triggered internally at  /opt/conda/conda-bld/pytorch_1614378124864/work/torch/csrc/utils/python_arg_parser.cpp:1005.)
  distmat.addmm_(1, -2, x, self.centers.t())


Test: [ 0/57]	Time  1.552 ( 1.552)	ent_Loss 4.3390e-01 (4.3390e-01)	center_loss 2.5113e+03 (2.5113e+03)	loss 7.9678e+00 (7.9678e+00)	Acc@1  87.50 ( 87.50)	Acc@5  96.88 ( 96.88)
Test: [ 5/57]	Time  0.996 ( 1.024)	ent_Loss 4.0481e-01 (5.0927e-01)	center_loss 2.3015e+03 (2.3159e+03)	loss 7.3093e+00 (7.4569e+00)	Acc@1  93.75 ( 85.42)	Acc@5  96.88 ( 96.35)
Test: [10/57]	Time  0.992 ( 1.012)	ent_Loss 6.7870e-01 (5.8212e-01)	center_loss 2.3938e+03 (2.3344e+03)	loss 7.8602e+00 (7.5852e+00)	Acc@1  75.00 ( 82.39)	Acc@5 100.00 ( 97.16)
Test: [15/57]	Time  1.023 ( 0.997)	ent_Loss 6.2946e-01 (5.9904e-01)	center_loss 2.4535e+03 (2.3481e+03)	loss 7.9899e+00 (7.6434e+00)	Acc@1  87.50 ( 82.81)	Acc@5  96.88 ( 97.46)
Test: [20/57]	Time  0.936 ( 0.992)	ent_Loss 5.5113e-01 (5.9357e-01)	center_loss 2.2764e+03 (2.3293e+03)	loss 7.3804e+00 (7.5815e+00)	Acc@1  75.00 ( 82.59)	Acc@5 100.00 ( 97.92)
Test: [25/57]	Time  0.928 ( 0.986)	ent_Loss 7.3436e-01 (6.0949e-01)	center_loss 2.2519e+03 (2.3437e+03)	loss 7.4901

Test: [170/450]	Time  0.300 ( 0.317)	ent_Loss 3.6976e-01 (6.0112e-01)	center_loss 1.9859e+03 (1.9842e+03)	loss 6.3276e+00 (6.5537e+00)	Acc@1  87.50 ( 83.33)	Acc@5 100.00 ( 97.28)
Test: [175/450]	Time  0.316 ( 0.317)	ent_Loss 1.7920e-01 (5.9848e-01)	center_loss 1.6815e+03 (1.9785e+03)	loss 5.2236e+00 (6.5339e+00)	Acc@1  96.88 ( 83.38)	Acc@5 100.00 ( 97.28)
Test: [180/450]	Time  0.292 ( 0.317)	ent_Loss 1.0581e+00 (6.0164e-01)	center_loss 1.9655e+03 (1.9755e+03)	loss 6.9545e+00 (6.5282e+00)	Acc@1  59.38 ( 83.17)	Acc@5  96.88 ( 97.31)
Test: [185/450]	Time  0.310 ( 0.318)	ent_Loss 3.7077e-01 (6.0265e-01)	center_loss 1.9732e+03 (1.9772e+03)	loss 6.2903e+00 (6.5342e+00)	Acc@1  84.38 ( 83.01)	Acc@5 100.00 ( 97.28)
Test: [190/450]	Time  0.327 ( 0.318)	ent_Loss 5.8647e-01 (6.0532e-01)	center_loss 1.8543e+03 (1.9750e+03)	loss 6.1492e+00 (6.5304e+00)	Acc@1  84.38 ( 82.94)	Acc@5 100.00 ( 97.28)
Test: [195/450]	Time  0.312 ( 0.318)	ent_Loss 1.8766e-01 (6.0043e-01)	center_loss 2.1877e+03 (1.9793e+03)

Test: [400/450]	Time  0.352 ( 0.330)	ent_Loss 1.7961e-01 (5.9586e-01)	center_loss 3.2892e+03 (2.3446e+03)	loss 1.0047e+01 (7.6296e+00)	Acc@1  93.75 ( 83.30)	Acc@5 100.00 ( 97.55)
Test: [405/450]	Time  0.331 ( 0.330)	ent_Loss 4.6967e-01 (5.9295e-01)	center_loss 3.4071e+03 (2.3549e+03)	loss 1.0691e+01 (7.6576e+00)	Acc@1  87.50 ( 83.41)	Acc@5 100.00 ( 97.56)
Test: [410/450]	Time  0.340 ( 0.330)	ent_Loss 7.0495e-02 (5.9163e-01)	center_loss 2.7834e+03 (2.3644e+03)	loss 8.4206e+00 (7.6848e+00)	Acc@1 100.00 ( 83.48)	Acc@5 100.00 ( 97.57)
Test: [415/450]	Time  0.310 ( 0.330)	ent_Loss 4.0284e-01 (5.8895e-01)	center_loss 2.8681e+03 (2.3728e+03)	loss 9.0070e+00 (7.7074e+00)	Acc@1  90.62 ( 83.59)	Acc@5  96.88 ( 97.57)
Test: [420/450]	Time  0.367 ( 0.330)	ent_Loss 3.8021e-01 (5.8702e-01)	center_loss 3.2144e+03 (2.3837e+03)	loss 1.0023e+01 (7.7380e+00)	Acc@1  90.62 ( 83.67)	Acc@5 100.00 ( 97.57)
Test: [425/450]	Time  0.350 ( 0.331)	ent_Loss 8.5685e-01 (5.8779e-01)	center_loss 3.3108e+03 (2.3926e+03)

Reached top1 accuracy of $83.17\%$ on the 